# Ant-Colony-Optimization Problem

This Jupyter Notebook deals with Ant-Colony-Optimization. All information is taken from Daniel Merkle's and Martin Middendorf's Chapter 14 on Swarm Intelligence in Search Methodologies. Introductory Tutorials in Optimization and Decision Support Techniques by Edmund K. Burke and Graham Kendall.

Ant-Colony Optimization is one of two main areas of Swarm Intelligence. It is used to solve optimization and search problems. The idea is that ant colonies find the shortest way form their nest to the food source with the help of pheromones. Experiments have shown that if you connect the nest with two different branches to a food source - one way is longer than the other one, ants will quickly find out which way is the shortest one. The result is an ant trail.

How exactly does it work? At the beginning both branches are taken with the same probability. After a while the pheromone concentration will be higher at the shorter branch since the ants are back at the nest way faster if they have taken the short road. The pheromone concentration gets higher more easily and since ants are taking the road with a higher pheromone concentration, eventually all ants take the short road

In [3]:
import pandas as pd
import numpy as np

## 1. Initialization
Importing .txt files and saving them as arrays.

In [56]:
def initialize(filepath): 
    with open(filepath, 'r') as t:
        distance_matrix = []
        for line in t.readlines():
            y = [int(value) for value in line.split()]
            distance_matrix.append(y)
        
        t.close()
        distance_matrix = np.array(distance_matrix)
    
    return distance_matrix

## 2. Solution Construction

In [67]:
def solution_construction(distance_matrix, pheromone_matrix, number_ants, alpha, beta):
    
    # test values
    #pheromone_matrix = [[0,0.4,0.1,1.2,0.7],[0.8,0,0.1,1.5,1.7],[0.9,0.5,0,0.2,0.7],[0.1,0.8,1.1,0,0.7],[0.5,0.4,0.1,1.2,0],[0,0.4,0.1,1.2,0.7]]
    #number_ants = 10
    #distance_matrix = [[0,1,5,2,6],[1,0,3,7,4],[5,3,0,3,5],[2,7,3,0,1],[6,4,5,1,0]]
    #alpha = 0.5
    #beta = 0.5

    ant_number = 0
    solution_matrix = []
    
    # creating a solution matrix corresponding to the size of the distance_matrix
    for i in range(number_ants):
        solution_matrix.append([])
        for j in range(len(distance_matrix)):
            solution_matrix[i].append([])
    
    # do for each ant
    while ant_number < number_ants:
        print('solution_construction')
        
        # initialize selection_set, append a placeholder '1' for each city available
        selection_set = []            
        selection_set = [[1] for i in range(len(distance_matrix))]
        
        # choose a random starting city, save it in the solution matrix and set it to '0' in selection_set
        start = (np.random.random_integers(0, len(distance_matrix)-1))
        solution_matrix[ant_number][0] = start
        selection_set[start] = 0
        
        # iterating through all cities left
        for x in range (0, len(distance_matrix)-1):
            probability = []                
                
            # create probability matrix of size of distance_matrix    
            for i in range (0, len(distance_matrix)):
                probability.append([])
                
                # checking if there is still a city left
                if((selection_set[i])!=0):
                    # calculating heuristics
                    variable = solution_matrix[ant_number][x]
                    
                    if alpha == 1 and beta == 1:
                        t= (pheromone_matrix[variable][i])*alpha
                        n = (1/(distance_matrix[variable][i]))*beta
                        sum = 0
                        
                        for j in range (0, len(distance_matrix)):
                            if((selection_set[j])!=0):
                                sum += ((pheromone_matrix[solution_matrix[ant_number][x]][j])*alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j])*beta)
                        
                    elif alpha == 1 and beta == 0:
                        t = (pheromone_matrix[variable][i])*alpha
                        n = (1/(distance_matrix[variable][i]))
                        
                        for j in range (0, len(distance_matrix)):
                            if((selection_set[j])!=0):
                                sum = ((pheromone_matrix[solution_matrix[ant_number][x]][j])*alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j]))
                        
                    else:
                        t = (pheromone_matrix[variable][i])**alpha
                        n = (1/(distance_matrix[variable][i]))**beta
                        
                        for j in range (0, len(distance_matrix)):
                            if((selection_set[j])!=0):
                                sum += ((pheromone_matrix[solution_matrix[ant_number][x]][j])**alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j])**beta)
                    
                    probability[i] = (t*n)/sum
                else:
                    probability[i]= 0
                    

            boundaries = []
            boundaries.append(0)
            boundaries[0] = probability[0]
 
            # calculation of boundaries for roulette-wheel-like selection of next city
            for i in range(1,len(probability)):
                boundaries.append(0)
                boundaries[i] = probability[i]+boundaries[i-1]

            # selection of the next city
            rand_mate = np.random.uniform(0,1)
            for m in range(len(boundaries)):
                if boundaries[m] > rand_mate:
                    city = m
                    break
                    
            solution_matrix[ant_number][x+1] = city
            selection_set[m] = 0
            
        ant_number += 1
        
    return solution_matrix

## 3. Evaporation

In [60]:
def evaporate(pheromone_matrix,evaporator):
    pheromone_matrix = pheromone_matrix * evaporator

    return pheromone_matrix

## 4. Intensification

solution_matrix will be a list of lists which each holding the numbers of the cities in the order that  each of the ants visited them


In [70]:
array2=[[1, 1, 1],[1, 1, 1],[1, 1, 1]]
array3=[[1, 2, 0],[0, 2, 1]]
array0=[[0, 4, 1],[1, 0, 5],[4, 1, 0]]

def intensification(distance_matrix, pheromone_matrix, solution_matrix,intensifier):
      
    best_solution = get_best_solution(distance_matrix, solution_matrix)
   
    for k in range(len(best_solution)):  
        x = best_solution[k]
        if k < len(best_solution)-1: 
            pheromone_matrix[x][best_solution[k+1]]= pheromone_matrix[x][best_solution[k+1]]*intensifier
        else:
            pheromone_matrix[best_solution[k]][0]= pheromone_matrix[best_solution[k]][0]*intensifier
   
    return pheromone_matrix
            
print(intensification(array0,array2,array3,1.05))

0 1
1 0
[[1, 1, 1.05], [1.05, 1, 1], [1, 1.05, 1]]


In [69]:
def get_best_solution(distance_matrix, solution_matrix):
    shortest_distance = 10000000000
    for i in solution_matrix:
        distance = 0
       
       
        for j in range(len(i)):
            y=i[j]
            if j < len(i)-1: 
                distance = distance + distance_matrix[y][i[j+1]]
            else:
                print(i[j],i[0])
                distance= distance + distance_matrix[i[j]][i[0]]
         
        
        if distance < shortest_distance:
            shortest_distance = distance
            best_solution = i
            #print(best_solution)
    return best_solution

## Action

In [68]:
# Choose the city matrix: '01_tsp.txt', '02_tsp.txt', '03_tsp.txt'
filepath = '01_tsp.txt'
distance_matrix = initialize(filepath)

# Set up pheromone matrix with shape 150 x 150 and fixed values = 0.01
pheromone_matrix = np.full(distance_matrix.shape, 0.01)

# Set number of ants
number_ants = 10

# Set alpha and beta (test: alpha = 1, beta = 0 AND alpha = 1, beta = 0)
alpha = 1
beta = 0

intensifier = 1.05
evaporator = 0.1

# "Measure" convergence

converged = False
count = 0
totalCount = 0

while not converged: 
    # solution_paths store the order how each ant visited the cities (each row is one ant)
    solution_matrix = solution_construction(distance_matrix, pheromone_matrix, number_ants, alpha, beta)
    print("solution_contruction done!")
    
    # Set up new pheromone matrix
    pheromone_matrix = evaporate(pheromone_matrix,evaporator)

    # Intensify good solutions and store them in a new pheromone matrix
    pheromone_matrix = intensification(distance_matrix, pheromone_matrix, solution_matrix,intensifier)
        
    # Convergence-Criterium:
    if count == 10:
    # if best_solution did not change, the algorithm converged, otherwise we reset the count value
        if best_solution_first == get_best_solution(distance_matrix, solution_matrix):
            converged = True
        else:
            count = 0
    # store the first best_solution for convergence-comparison
    elif count == 0:
        best_solution_first = get_best_solution(distance_matrix, solution_matrix)
        count += 1
    else:
    # sincrease count
        count += 1
    totalCount += 1
    print(totalCount)
    print(get_best_solution(distance_matrix, solution_matrix))

solution_construction


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: This function is deprecated. Please call randint(0, 149 + 1) instead


solution_construction
solution_construction
solution_construction
solution_construction
solution_construction
solution_construction
solution_construction
solution_construction
solution_construction
21 0
0 1
1 2
2 5
5 4
4 3
3 6
6 8
8 9
9 7
7 10
10 11
11 12
12 13
13 15
15 14
14 16
16 18
18 17
17 19
19 20
20 22
22 23
23 25
25 24
24 26
26 27
27 30
30 29
29 28
28 31
31 32
32 33
33 34
34 35
35 37
37 36
36 38
38 39
39 40
40 41
41 42
42 43
43 45
45 46
46 44
44 47
47 48
48 52
52 49
49 50
50 51
51 53
53 54
54 55
55 57
57 61
61 58
58 56
56 60
60 62
62 63
63 59
59 64
64 65
65 66
66 67
67 68
68 69
69 70
70 72
72 71
71 73
73 74
74 77
77 75
75 76
76 78
78 79
79 80
80 82
82 81
81 83
83 84
84 86
86 85
85 87
87 88
88 90
90 89
89 92
92 93
93 94
94 91
91 95
95 96
96 98
98 97
97 99
99 100
100 101
101 102
102 103
103 104
104 111
111 105
105 106
106 107
107 108
108 109
109 110
110 112
112 114
114 113
113 115
115 117
117 118
118 116
116 119
119 120
120 121
121 123
123 122
122 124
124 126
126 125
125 127
127 1

146 148
148 147
147 149
149 3
28334
147 1
1 0
0 2
2 4
4 3
3 5
5 6
6 8
8 7
7 9
9 10
10 11
11 12
12 13
13 15
15 14
14 16
16 18
18 17
17 19
19 20
20 22
22 21
21 23
23 24
24 25
25 26
26 27
27 29
29 28
28 30
30 32
32 31
31 34
34 33
33 35
35 37
37 36
36 38
38 39
39 40
40 41
41 42
42 43
43 45
45 48
48 46
46 44
44 47
47 50
50 51
51 49
49 52
52 53
53 54
54 55
55 56
56 57
57 62
62 58
58 59
59 61
61 60
60 63
63 64
64 65
65 67
67 66
66 68
68 69
69 70
70 72
72 71
71 73
73 75
75 74
74 78
78 76
76 77
77 80
80 79
79 81
81 83
83 84
84 85
85 82
82 86
86 87
87 88
88 89
89 90
90 91
91 92
92 94
94 95
95 93
93 96
96 97
97 98
98 100
100 99
99 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 112
112 111
111 114
114 115
115 113
113 116
116 117
117 118
118 119
119 120
120 121
121 122
122 124
124 123
123 125
125 126
126 128
128 127
127 129
129 130
130 131
131 133
133 132
132 134
134 136
136 135
135 137
137 139
139 138
138 140
140 141
141 142
142 143
143 144
144 145
145 146
146 149


95 96
96 91
91 97
97 99
99 100
100 98
98 101
101 102
102 103
103 104
104 105
105 107
107 106
106 108
108 109
109 110
110 111
111 112
112 114
114 115
115 116
116 113
113 117
117 119
119 118
118 120
120 121
121 123
123 122
122 124
124 126
126 125
125 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
135 136
136 137
137 139
139 138
138 140
140 141
141 142
142 143
143 144
144 145
145 146
146 148
148 147
147 149
149 3
28334
147 1
1 0
0 2
2 4
4 3
3 5
5 6
6 8
8 7
7 9
9 10
10 11
11 12
12 13
13 15
15 14
14 16
16 18
18 17
17 19
19 20
20 22
22 21
21 23
23 24
24 25
25 26
26 27
27 29
29 28
28 30
30 32
32 31
31 34
34 33
33 35
35 37
37 36
36 38
38 39
39 40
40 41
41 42
42 43
43 45
45 48
48 46
46 44
44 47
47 50
50 51
51 49
49 52
52 53
53 54
54 55
55 56
56 57
57 62
62 58
58 59
59 61
61 60
60 63
63 64
64 65
65 67
67 66
66 68
68 69
69 70
70 72
72 71
71 73
73 75
75 74
74 78
78 76
76 77
77 80
80 79
79 81
81 83
83 84
84 85
85 82
82 86
86 87
87 88
88 89
89 90
90 91
91 92
92 94
94 95
95 93
93

...

28268
72 0
0 1
1 3
3 2
2 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 16
16 15
15 17
17 18
18 20
20 19
19 22
22 23
23 24
24 21
21 25
25 26
26 27
27 29
29 30
30 28
28 31
31 32
32 33
33 34
34 36
36 37
37 35
35 38
38 39
39 40
40 41
41 42
42 43
43 45
45 47
47 44
44 46
46 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 57
57 61
61 56
56 58
58 59
59 60
60 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 73
73 71
71 74
74 77
77 75
75 76
76 78
78 80
80 79
79 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 90
90 89
89 92
92 91
91 93
93 94
94 97
97 95
95 96
96 98
98 99
99 100
100 102
102 101
101 103
103 104
104 112
112 107
107 106
106 105
105 108
108 109
109 110
110 111
111 113
113 114
114 115
115 117
117 118
118 116
116 119
119 120
120 121
121 122
122 123
123 125
125 124
124 126
126 129
129 127
127 128
128 130
130 131
131 132
132 133
133 135
135 134
134 136
136 137
137 139
139 140
140 138
138 141
141 142
142 143
143 144
144 145
145 146
146 149
149 147
147 148
148 72
27252
121 0
0 

82 83
83 84
84 87
87 86
86 85
85 88
88 89
89 90
90 91
91 92
92 94
94 97
97 95
95 96
96 98
98 100
100 99
99 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 114
114 115
115 116
116 113
113 117
117 118
118 119
119 120
120 121
121 124
124 125
125 122
122 123
123 126
126 129
129 127
127 128
128 130
130 131
131 133
133 132
132 134
134 135
135 136
136 137
137 139
139 138
138 141
141 140
140 142
142 143
143 144
144 145
145 146
146 149
149 147
147 148
148 93
27876
61 0
0 1
1 2
2 5
5 3
3 4
4 6
6 9
9 8
8 7
7 10
10 11
11 12
12 13
13 14
14 15
15 16
16 18
18 19
19 17
17 20
20 21
21 23
23 22
22 24
24 25
25 26
26 27
27 29
29 30
30 28
28 31
31 32
32 33
33 34
34 35
35 37
37 36
36 38
38 39
39 40
40 41
41 42
42 43
43 45
45 44
44 46
46 47
47 49
49 48
48 51
51 50
50 53
53 52
52 55
55 57
57 60
60 56
56 54
54 59
59 62
62 63
63 58
58 64
64 65
65 67
67 66
66 68
68 69
69 70
70 73
73 71
71 72
72 74
74 75
75 76
76 78
78 77
77 79
79 80
80 82
82 81
81 83
83 84
84 87
87

24 25
25 26
26 28
28 27
27 30
30 29
29 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 48
48 50
50 47
47 46
46 49
49 51
51 53
53 52
52 55
55 56
56 57
57 62
62 54
54 58
58 59
59 60
60 61
61 64
64 63
63 65
65 66
66 68
68 69
69 67
67 70
70 72
72 71
71 73
73 75
75 74
74 77
77 78
78 76
76 80
80 79
79 81
81 83
83 84
84 82
82 85
85 86
86 88
88 87
87 90
90 89
89 92
92 93
93 94
94 96
96 91
91 95
95 97
97 99
99 98
98 100
100 101
101 102
102 103
103 104
104 107
107 106
106 105
105 109
109 108
108 110
110 111
111 112
112 114
114 115
115 117
117 116
116 113
113 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 128
128 127
127 129
129 130
130 131
131 133
133 132
132 134
134 135
135 136
136 137
137 138
138 139
139 140
140 141
141 142
142 143
143 144
144 145
145 146
146 149
149 147
147 148
148 9
26318
26318
108 0
0 1
1 2
2 4
4 3
3 5
5 6
6 8
8 9
9 7
7 10
10 12
12 11
11 13
13 15
15 16
16 14
14 17
17 18
18 20
20 19
19 21
21 23
23 22
22 24
24

27 30
30 29
29 31
31 28
28 32
32 33
33 34
34 35
35 37
37 36
36 39
39 40
40 38
38 41
41 42
42 43
43 45
45 48
48 46
46 44
44 47
47 50
50 49
49 51
51 52
52 54
54 53
53 55
55 57
57 60
60 56
56 58
58 59
59 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 79
79 76
76 75
75 77
77 78
78 80
80 82
82 81
81 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 94
94 97
97 93
93 95
95 96
96 98
98 100
100 99
99 102
102 101
101 103
103 104
104 112
112 107
107 105
105 106
106 109
109 110
110 111
111 114
114 115
115 117
117 113
113 116
116 118
118 119
119 120
120 121
121 123
123 124
124 126
126 125
125 122
122 127
127 128
128 129
129 130
130 131
131 133
133 132
132 134
134 135
135 136
136 137
137 139
139 138
138 141
141 140
140 142
142 143
143 144
144 145
145 146
146 149
149 147
147 148
148 108
26944
88 0
0 1
1 2
2 4
4 3
3 5
5 6
6 8
8 7
7 9
9 10
10 11
11 12
12 13
13 15
15 16
16 14
14 17
17 18
18 19
19 20
20 21
21 22
22 23
23 25
25 24
24 26
26 27
27 28
28 29
29 

KeyboardInterrupt: 